In [1]:
import pandas as pd 
import numpy as np
from scipy import linalg
from scipy.spatial.distance import pdist
import networkx as nx
import sys
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import DistanceMetric
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
from adjustText import adjust_text
import seaborn as sns
from matplotlib import rcParams, cycler

In [2]:
def norm_and_pca_from_df(df):
    '''
    Function processes the data using Principal component analysis(PCA)
    Args:
        df : Dataframe 
    Returns dataframe containing rincipal component of df
    '''
    
    df_normalized = pd.DataFrame()
    cols = list(df.columns)
    df_normalized[cols] = pd.DataFrame(normalize(StandardScaler().fit_transform(df[cols])))
    
    df_pca = pd.DataFrame(PCA(n_components = 2).fit_transform(df_normalized))
    df_pca.columns = ['P1', 'P2']
    df_pca.index = df.index
    
    return df_normalized, df_pca

########################################################################################

def calculate_eigen_self_tuning(df, k):
    
    dimension = df.shape[0]
    dist_ = pdist(df)
    pd = np.zeros([dimension, dimension])
    dist = iter(dist_)
    for i in range(dimension):
        for j in range(i+1, dimension):  
            d = next(dist)
            pd[i,j] = d
            pd[j,i] = d
            
    #calculate local sigma
    sigmas = np.zeros(dimension)
    for i in tqdm(range(len(pd))):
        sigmas[i] = sorted(pd[i])[7]
    
    adjacency_matrix = np.zeros([dimension, dimension])
    dist = iter(dist_)
    for i in tqdm(range(dimension)):
        for j in range(i+1, dimension):  
            d = np.exp(-1*next(dist)**2/(sigmas[i]*sigmas[j]))
            adjacency_matrix[i,j] = d
            adjacency_matrix[j,i] = d
            
    degree_matrix = np.sum(adjacency_matrix, axis=0) * np.eye(dimension)
    
    # Normalized laplacian matrix
    d_half = linalg.fractional_matrix_power(degree_matrix, -0.5)
    laplacian_matrix_normalized = np.matmul(np.matmul(d_half, adjacency_matrix), d_half)
    
    e, v = np.linalg.eigh(laplacian_matrix_normalized)
    X = v[:, -1*k:]
    
    row_sums = X.sum(axis=1)
    Y = X / row_sums[:, np.newaxis]
            
    return X, Y

############################################################################################

def ensemble_classification(df, test_index_list, class_label_list,n):
    
    df['class_label'] = class_label_list
    
    train_df = df.loc[~ df.index.isin(test_index_list)]
    test_df = df.loc[df.index.isin(test_index_list)]

    X_train = train_df.drop(['class_label'],axis=1)
    y_train = train_df[['class_label']]

    X_test = test_df.drop(['class_label'],axis=1)
    y_test = test_df[['class_label']]
    
    acc_1 = [] 
    mcc_1 = []
    acc_2 = []
    mcc_2 = []
    acc_3 = []
    mcc_3 = []
    
    for values in range(0,n+1):
        
        classifier0 = NearestCentroid()
        classifier0.fit(X_train, y_train.values.ravel())
        acc_1.append(classifier0.score(X_test, y_test))
        mcc_1.append(mcc(classifier0.predict(X_test), y_test))
        
        classifier1 = KNeighborsClassifier(n_neighbors=5)
        classifier1.fit(X_train, y_train.values.ravel())
        acc_2.append(classifier1.score(X_test, y_test))
        mcc_2.append(mcc(classifier1.predict(X_test), y_test))
        
        classifier2 = DecisionTreeClassifier(criterion = "gini")
        classifier2.fit(X_train, y_train)
        acc_3.append(classifier2.score(X_test, y_test))
        mcc_3.append(mcc(classifier2.predict(X_test), y_test))
        
    print('Nearest Centroid (acc, mcc) -',sum(acc_1)/len(acc_1), sum(mcc_1)/len(mcc_1))
    print('K nearest neighbours (acc, mcc) -',sum(acc_2)/len(acc_2),sum(mcc_2)/len(mcc_2) )
    print('Decision Tree (acc, mcc) -',sum(acc_3)/len(acc_3),sum(mcc_3)/len(mcc_3) )
    
    
############################################################################################

In [ ]:
# Load dataset 1
df_csv_data = pd.read_csv("dataset_1/d1_mp.csv", index_col = 0)
df_normalized, df_pca = norm_and_pca_from_df(df_csv_data)

df_pca['class_label'] = ['CD' if 'CD' in index 
                         else 'IBS' if 'IBS' in  index
                         else 'UCr' if 'UCr' in index 
                         else 'UCa' if 'UCa' in index 
                         else 'GCA' if 'GCA' in index 
                         else 'A' if 'C' not in index else 'C' 
                         for index, patient in df_pca.iterrows()]
df_pca['class_label'] = df_pca['class_label'].astype('category').cat.codes

df_pca_control = df_pca[['P1','P2','P3']]

In [ ]:
## Load dataset 2
df_csv_data = pd.read_csv("dataset_2/d2_mp.csv", index_col=0)
# df_csv_data = df_csv_data.iloc[:, 0:12649]

df_csv_data2 = df_csv_data.loc[:,(df_csv_data.mean() >= 0.5)]
df_normalized, df_pca = norm_and_pca_from_df(df_csv_data)

df_pca['class_label'] = ['C' if 'C' in index else 'N' if 'N' in index else 'H' for index, patient in df_pca.iterrows()]
df_pca['class_label'] = df_pca['class_label'].astype('category').cat.codes
df_pca_control = df_pca[['P1','P2']]

In [3]:
## Load combined dataset
df_csv = pd.read_csv("d3_mp.csv", index_col=0)
df_csv_data = df_csv.iloc[:, 0:170]

df_normalized, df_pca = norm_and_pca_from_df(df_csv_data)

df_pca['class_label'] = ['N' if 'N' in index
                         else 'H' if 'H' in index
                         else 'CD' if 'CD' in index 
                         else 'IBS' if 'IBS' in  index
                         else 'UCr' if 'UCr' in index 
                         else 'UCa' if 'UCa' in index
                         else 'GCA' if 'GCA' in index 
                         else 'A' if '_A' in index else 'C' 
                         for index, patient in df_pca.iterrows()]

df_pca['class_label'] = df_pca['class_label'].astype('category').cat.codes

C:\Users\49171\AppData\Local\Temp\ipykernel_16676\1843745508.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_normalized[cols] = pd.DataFrame(normalize(StandardScaler().fit_transform(df[cols])))
C:\Users\49171\AppData\Local\Temp\ipykernel_16676\1843745508.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_normalized[cols] = pd.DataFrame(normalize(StandardScaler().fit_transform(df[cols])))
C:\Users\49171\AppData\Local\Temp\ipykernel_16676\1843745508.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is u

In [ ]:
df_pca.class_label.value_counts()

In [4]:
# index_list = ['P13_C','P14_C','P15_C','P16_C','P17_C',
#               'P25_CD','P26_CD','P27_CD',
#               'P34_UCr',
#               'P41_UCa', 'P42_UCa',
#               'P52_IBS', 'P53_IBS', 'P54_IBS','P55_IBS',
#               'P62_A', 'P63_A',
#               'P70_GCA', 'P71_GCA']

# index_list = ['C_15','C_16','C_17','C_18','C_19',
#               'N_44','N_45','N_46','N_47','N_48','N_49','N_50','N_51',
#               'H_74','H_75','H_76','H_77','H_78','H_79','H_80']

index_list = ['P13_C','P14_C','P15_C','P16_C','P17_C',
              'P25_CD','P26_CD','P27_CD',
              'P34_UCr',
              'P41_UCa', 'P42_UCa',
              'P52_IBS', 'P53_IBS', 'P54_IBS','P55_IBS',
              'P62_A', 'P63_A',
              'P70_GCA', 'P71_GCA',
              'C_15','C_16','C_17','C_18','C_19',
              'N_44','N_45','N_46','N_47','N_48','N_49','N_50','N_51',
              'H_74','H_75','H_76','H_77','H_78','H_79','H_80']

In [ ]:
ensemble_classification(df_csv_data, index_list, df_pca.class_label, 30)

In [ ]:
df_normalized.index = df_csv_data.index
ensemble_classification(df_normalized, index_list, df_pca.class_label, 30)

In [7]:
Xst, Yst = calculate_eigen_self_tuning(df_normalized, 18)
df_U = pd.DataFrame(Xst)
df_N = pd.DataFrame(Yst)

df_U.index = df_csv_data.index
ensemble_classification(df_U, index_list, df_pca.class_label, 30)

100%|██████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 2335.72it/s]


Nearest Centroid (acc, mcc) - 0.3076923076923077 0.20089040639258376
K nearest neighbours (acc, mcc) - 0.5128205128205128 0.4056977078868017
Decision Tree (acc, mcc) - 0.3961952026468157 0.2759428597136052


In [ ]:
df_pca.class_label.shape